<a href="https://colab.research.google.com/github/irawan09/PySpark/blob/main/PySpark_Hive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Environment Setup

In [5]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz
!tar xf spark-3.0.1-bin-hadoop2.7.tgz
!pip install -q findspark
!git clone 'https://github.com/irawan09/PySpark.git'
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop2.7"
import findspark
findspark.init()

Cloning into 'PySpark'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 22 (delta 5), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (22/22), done.


In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Learning_Spark") \
    .getOrCreate()

sc = spark.sparkContext
lines = sc.textFile("spark-3.0.1-bin-hadoop2.7/README.md")

In [4]:
input = sc.textFile("spark-2.4.4-bin-hadoop2.7/*.md")

In [11]:
data = spark.read.csv('/content/PySpark/Matches.csv',inferSchema=True, header=True)

In [20]:
data.printSchema()

root
 |-- Match_ID: string (nullable = true)
 |-- Div: string (nullable = true)
 |-- Season: integer (nullable = true)
 |-- Date: string (nullable = true)
 |-- HomeTeam: string (nullable = true)
 |-- AwayTeam: string (nullable = true)
 |-- FTHG: integer (nullable = true)
 |-- FTAG: integer (nullable = true)
 |-- FTR: string (nullable = true)



In [21]:
data.show(5)

+--------+---+------+---------+-------------+--------------+----+----+---+
|Match_ID|Div|Season|     Date|     HomeTeam|      AwayTeam|FTHG|FTAG|FTR|
+--------+---+------+---------+-------------+--------------+----+----+---+
|       1| D2|  2009| 4/4/2010|   Oberhausen|Kaiserslautern|   2|   1|  H|
|       2| D2|  2009|11/1/2009|  Munich 1860|Kaiserslautern|   0|   1|  A|
|       3| D2|  2009|10/4/2009|Frankfurt FSV|Kaiserslautern|   1|   1|  D|
|       4| D2|  2009|2/21/2010|Frankfurt FSV|     Karlsruhe|   2|   1|  H|
|       5| D2|  2009|12/6/2009|        Ahlen|     Karlsruhe|   1|   3|  A|
+--------+---+------+---------+-------------+--------------+----+----+---+
only showing top 5 rows



In [23]:
data.count()

24627

In [25]:
data.describe().show()

+-------+-------------------+--------------------+----------------+--------+--------+--------+------------------+------------------+-----+
|summary|           Match_ID|                 Div|          Season|    Date|HomeTeam|AwayTeam|              FTHG|              FTAG|  FTR|
+-------+-------------------+--------------------+----------------+--------+--------+--------+------------------+------------------+-----+
|  count|              24626|               24626|           24625|   24625|   24625|   24625|             24625|             24625|24625|
|   mean| 27335.103228426397|                null|2004.69876142132|    null|    null|    null|1.5170355329949239| 1.097502538071066| null|
| stddev|  16631.84606173585|                null|7.16693132762853|    null|    null|    null|1.3594706607115317|1.1635103573988272| null|
|    min|"{""mode"":""full""|""isActive"":false}"|            1993|1/1/1994|  Aachen|  Aachen|                -1|                -1|    A|
|    max|                99

In [27]:
data.select("Season").distinct().show()

+------+
|Season|
+------+
|  2003|
|  2007|
|  2015|
|  2006|
|  2013|
|  null|
|  1997|
|  1994|
|  2014|
|  2004|
|  1996|
|  1998|
|  2012|
|  2009|
|  2016|
|  1995|
|  2001|
|  2005|
|  2000|
|  2010|
+------+
only showing top 20 rows



In [24]:
dataF=data.select("Div","Season","HomeTeam","AwayTeam").show(5)

+---+------+-------------+--------------+
|Div|Season|     HomeTeam|      AwayTeam|
+---+------+-------------+--------------+
| D2|  2009|   Oberhausen|Kaiserslautern|
| D2|  2009|  Munich 1860|Kaiserslautern|
| D2|  2009|Frankfurt FSV|Kaiserslautern|
| D2|  2009|Frankfurt FSV|     Karlsruhe|
| D2|  2009|        Ahlen|     Karlsruhe|
+---+------+-------------+--------------+
only showing top 5 rows



In [28]:
from pyspark.sql import functions as F

data.groupBy("HomeTeam").agg(F.sum("FTHG")).show()

+--------------+---------+
|      HomeTeam|sum(FTHG)|
+--------------+---------+
|         Mainz|      651|
|Kaiserslautern|      674|
|     Tottenham|      772|
|    Schalke 04|      676|
|      Nurnberg|      555|
|        Bolton|      319|
|    M'Gladbach|       38|
|    Reutlingen|      100|
|           Ulm|       88|
|  Union Berlin|      280|
|Sheffield Weds|      200|
|    Sunderland|      336|
| Nott'm Forest|       98|
| Frankfurt FSV|      187|
|      Bradford|       46|
|   Schweinfurt|       23|
|          Hull|      107|
|        Siegen|       24|
|      Coventry|      190|
|       Swindon|       25|
+--------------+---------+
only showing top 20 rows



In [14]:
dataF.write.csv("Home_vs_Away.csv")

In [17]:
rddL=sc.textFile("Home_vs_Away.csv")
rddL.collect()

['D2,2009,Oberhausen,Kaiserslautern',
 'D2,2009,Munich 1860,Kaiserslautern',
 'D2,2009,Frankfurt FSV,Kaiserslautern',
 'D2,2009,Frankfurt FSV,Karlsruhe',
 'D2,2009,Ahlen,Karlsruhe',
 'D2,2009,Union Berlin,Karlsruhe',
 'D2,2009,Paderborn,Karlsruhe',
 'D2,2009,Bielefeld,Karlsruhe',
 'D2,2009,Kaiserslautern,Karlsruhe',
 'D2,2009,Hansa Rostock,Karlsruhe',
 'D2,2009,Greuther Furth,Karlsruhe',
 'D2,2009,Koblenz,Karlsruhe',
 'D2,2009,Oberhausen,Karlsruhe',
 'D2,2009,Cottbus,Karlsruhe',
 'D2,2009,Duisburg,Karlsruhe',
 'D2,2009,Fortuna Dusseldorf,Karlsruhe',
 'D2,2009,Aachen,Karlsruhe',
 'D2,2009,Augsburg,Karlsruhe',
 'D2,2009,St Pauli,Karlsruhe',
 'D2,2009,Munich 1860,Karlsruhe',
 'D1,2009,Bochum,Leverkusen',
 'D1,2009,Bayern Munich,Leverkusen',
 "D1,2009,M'gladbach,Leverkusen",
 'D1,2009,Mainz,Leverkusen',
 'D1,2009,Hamburg,Leverkusen',
 'D1,2009,Stuttgart,Leverkusen',
 'D1,2009,Dortmund,Leverkusen',
 'D1,2009,Schalke 04,Leverkusen',
 'D1,2009,Freiburg,Leverkusen',
 'D1,2009,Werder Bremen,Lev

#Apache Hive

In [18]:
from pyspark.sql import Row
from pyspark.sql import HiveContext

sqlContext = HiveContext(sc)
test_list = [('A', 25),('B', 20),('C', 25),('D', 18)]
rdd = sc.parallelize(test_list)
team = rdd.map(lambda x: Row(name=x[0], age=int(x[1])))
schemaTeam = sqlContext.createDataFrame(team)

# Register it as a temp table
sqlContext.registerDataFrameAsTable(schemaTeam, "test_table")
sqlContext.sql("show tables").show()

+--------+----------+-----------+
|database| tableName|isTemporary|
+--------+----------+-----------+
|        |test_table|       true|
+--------+----------+-----------+



In [29]:
type(rdd)

pyspark.rdd.RDD

In [19]:
sqlContext.sql("Select * from test_table").show()

+----+---+
|name|age|
+----+---+
|   A| 25|
|   B| 20|
|   C| 25|
|   D| 18|
+----+---+

